In [4]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
%matplotlib inline
# 한글폰트 사용
mpl.rc('font', family='Malgun Gothic')
mpl.rc('axes', unicode_minus=False)

In [ ]:
# 빈 데이터프레임을 만들어주고
starbucks = pd.DataFrame(columns=['상호명', '지점명', '도로명주소', '경도', '위도'])
coffeebean = pd.DataFrame(columns=['상호명', '지점명', '도로명주소', '경도', '위도'])
ediya = pd.DataFrame(columns=['상호명', '지점명', '도로명주소', '경도', '위도'])
paik = pd.DataFrame(columns=['상호명', '지점명', '도로명주소', '경도', '위도'])

In [6]:
# 17개 파일 모두
from glob import glob
data_files = glob('./data/소상공인시장진흥공단_상가(상권)정보*.csv')

for filename in data_files:
    print(filename)
    df = pd.read_csv(filename, sep='|')  #파일읽고
    df = df[df['상권업종중분류코드'] == 'Q12'] # 코드Q12만불러서
    df = df[['상호명', '지점명', '도로명주소', '경도', '위도']] #데이터프레임에

    # 컬럼상호명에 지정한str이 대소문자구별없이(case=False) 포함되어있다면, 각각변수에 저장
    sb = df[df['상호명'].str.contains('스타벅스|STARBUCKS|스타박스', case=False)]
    cb = df[df['상호명'].str.contains('커피빈|COFFEEBEAN', case=False)]
    ed = df[df['상호명'].str.contains('이디야|EDIYA|이디아', case=False)]
    bd = df[df['상호명'].str.contains('빽다방|PAIKSCOFFEE', case=False)]

    # 빈 데이터프레임과 각각 합치기concat
    starbucks = pd.concat([starbucks, sb])
    coffeebean = pd.concat([coffeebean, cb])
    ediya = pd.concat([ediya, ed])
    paik = pd.concat([paik, bd])

In [ ]:
# 결과저장
starbucks.to_csv('./results/스타벅스.csv', index=False)
coffeebean.to_csv('./results/커피빈.csv', index=False)
ediya.to_csv('./results/이디야.csv', index=False)
paik.to_csv('./results/빽다방.csv', index=False)

In [ ]:
# 데이터가져오기
starbucks = pd.read_csv('./results/스타벅스.csv')
coffeebean = pd.read_csv('./results/커피빈.csv')
ediya = pd.read_csv('./results/이디야.csv')
paik = pd.read_csv('./results/빽다방.csv')

In [ ]:
# mapUtil.py모듈활용 도로명주소에서 광역시도와 시군고/행정구명 찾기
import cartogram.mapUtil as mu

starbucks = mu.split_addr(starbucks)
coffeebean = mu.split_addr(coffeebean)
ediya = mu.split_addr(ediya)
paik = mu.split_addr(paik)

In [ ]:
# mapUtil.py모듈활용 도로명주소에서 광역시도와 시군고/행정구명 찾기-스타벅스
starbucks.reset_index(inplace=True)
starbucks.drop('index', axis=1, inplace=True)
starbucks['ID'] = mu.get_ID(starbucks)
starbucks['브랜드명'] = '스타벅스'
starbucks['스타벅스 매장수'] = 1
starbucks.head()

In [ ]:
# mapUtil.py모듈활용 도로명주소에서 광역시도와 시군고/행정구명 찾기-커피빈
coffeebean.reset_index(inplace=True)
coffeebean.drop('index', axis=1, inplace=True)
coffeebean['ID'] = mu.get_ID(coffeebean)
coffeebean['브랜드명'] = '커피빈'
coffeebean['커피빈 매장수'] = 1
coffeebean.head()

In [ ]:
# mapUtil.py모듈활용 도로명주소에서 광역시도와 시군고/행정구명 찾기-이디야
ediya.reset_index(inplace=True)
ediya.drop('index', axis=1, inplace=True)
ediya['ID'] = mu.get_ID(ediya)
ediya['브랜드명'] = '이디야'
ediya['이디야 매장수'] = 1
ediya.head()

In [ ]:
# mapUtil.py모듈활용 도로명주소에서 광역시도와 시군고/행정구명 찾기-빽다방
paik.reset_index(inplace=True)
paik.drop('index', axis=1, inplace=True)
paik['ID'] = mu.get_ID(paik)
paik['브랜드명'] = '빽다방'
paik['빽다방 매장수'] = 1
paik.head()

## 브랜드별 피벗 테이블 생성

In [ ]:
# 브랜드별 피벗 테이블 생성-스타벅스
pt_sb = starbucks.pivot_table('스타벅스 매장수', index='ID', aggfunc='sum')
pt_sb = pt_sb.reset_index()
print(len(pt_sb))
pt_sb.head()

In [ ]:
# 브랜드별 피벗 테이블 생성-커피빈
pt_cb = coffeebean.pivot_table('커피빈 매장수', index='ID', aggfunc='sum')
pt_cb = pt_cb.reset_index()
print(len(pt_cb))
pt_cb.head()

In [ ]:
# 브랜드별 피벗 테이블 생성-이디야
pt_ed = ediya.pivot_table('이디야 매장수', index='ID', aggfunc='sum')
pt_ed = pt_ed.reset_index()
print(len(pt_ed))
pt_ed.head()

In [ ]:
# 브랜드별 피벗 테이블 생성-빽다방
pt_bd = paik.pivot_table('빽다방 매장수', index='ID', aggfunc='sum')
pt_bd = pt_bd.reset_index()
print(len(pt_bd))
pt_bd.head()

## 카페 데이터 병합

In [ ]:
coffee = pd.merge(pt_ed, pt_sb, how='left', on='ID')  #Id를 기준으로 왼쪽으로 merge합침 이디야+스타벅스
coffee.head()

In [ ]:
coffee = pd.merge(coffee, pt_cb, how='left', on='ID') #Id를 기준으로 왼쪽으로 merge합침 이+스+커피빈
coffee = pd.merge(coffee, pt_bd, how='left', on='ID')  #Id를 기준으로 왼쪽으로 merge합침 이+스+빈+빽다방
coffee.head()

In [ ]:
coffee = coffee.fillna(0) #NaN값에 0을 넣어라
coffee.head()

In [ ]:
coffee['스타벅스 매장수'] = coffee['스타벅스 매장수'].astype(int)
coffee['커피빈 매장수'] = coffee['커피빈 매장수'].astype(int)
coffee['빽다방 매장수'] = coffee['빽다방 매장수'].astype(int)

In [ ]:
#컬럼커피지수 추가 
coffee['커피지수'] = (coffee['스타벅스 매장수']+coffee['커피빈 매장수']) / \
                        (coffee['이디야 매장수']+coffee['빽다방 매장수'])
coffee.tail()

In [ ]:
coffee.to_csv('./results/커피피봇테이블.csv', index=False)

## 카토그램과 커피 데이터 합치기

In [ ]:
#앞전 인구소멸위기지역 03.pop.csv
df = pd.read_csv('./results/03.pop.csv', index_col = 0)
df.head()

In [ ]:
#필요한 컬럼만
df = df[['ID', 'y', 'x']]
df.head()

In [ ]:
coffee_index = pd.merge(df, coffee, how='left', on='ID')
coffee_index = coffee_index.fillna(0)
coffee_index.head()

## 커피지수(카토그램)

In [ ]:
# drawKorea.py 모듈활용
import cartogram.drawKorea as dk
dk.drawKorea('커피지수', coffee_index, 'Reds')

In [ ]:
# 커피지수 Top 10
coffee_index.sort_values(by='커피지수', ascending=False).head(10)

In [ ]:
# 스타벅스 매장수
dk.drawKorea('스타벅스 매장수', coffee_index, 'Greens'

In [ ]:
# 스타벅스 매장수 Top 5
coffee_index.sort_values(by='스타벅스 매장수', ascending=False).head()

In [ ]:
#커피빈 매장수
dk.drawKorea('커피빈 매장수', coffee_index, 'Blues')

In [ ]:
# 커피빈 매장수 Top 5
coffee_index.sort_values(by='커피빈 매장수', ascending=False).head()

In [ ]:
# 이디야 매장수
dk.drawKorea('이디야 매장수', coffee_index, 'Oranges')

In [ ]:
# 이디야 매장수 Top 5
coffee_index.sort_values(by='이디야 매장수', ascending=False).head()

In [ ]:
# 빽다방 매장수
dk.drawKorea('빽다방 매장수', coffee_index, 'PuRd')

In [ ]:
# 빽다방 매장수 Top 5
coffee_index.sort_values(by='빽다방 매장수', ascending=False).head()

## 지도위에 표시하기

In [ ]:
import folium
import json

In [ ]:
coffee_index_folium = coffee_index.set_index('ID')
coffee_index_folium.head()

In [ ]:
geo_path = './data/skorea_municipalities_geo_simple.json'
geo_data = json.load(open(geo_path, encoding='utf8'))
#전국지도
map = folium.Map(location=[35.9902, 127.054], zoom_start=7)
folium.Choropleth(geo_data = geo_data,
               data = coffee_index_folium['커피지수'],
               columns = [coffee_index_folium.index, coffee_index_folium['커피지수']],
               fill_color = 'YlGnBu',
               key_on = 'feature.id').add_to(map)
title_html = '<h3 align="center" style="font-size:20px"><b>커피지수</b></h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map

In [ ]:
#전국지도
map = folium.Map(location=[35.9902, 127.054], zoom_start=7)
folium.Choropleth(geo_data = geo_data,
               data = coffee_index_folium['스타벅스 매장수'],
               columns = [coffee_index_folium.index, coffee_index_folium['스타벅스 매장수']],
               fill_color = 'Greens',
               key_on = 'feature.id').add_to(map)
title_html = '<h3 align="center" style="font-size:20px"><b>스타벅스 매장수</b></h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map